# Visualisations des vulnérabilités
Ce notebook génère l'ensemble des graphiques pour analyser les données issues de `data/consolidated.csv`.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Chargement et prétraitement
df = pd.read_csv('data\consolidated.csv', sep=',')
df['cvss_score'] = pd.to_numeric(df['cvss_score'], errors='coerce')
df['epss_score'] = pd.to_numeric(df['epss_score'], errors='coerce')
df['date_pub'] = pd.to_datetime(df['date_pub'], errors='coerce')

# Affichage d'un aperçu
df.head()

FileNotFoundError: [Errno 2] No such file or directory: './data/consolidated.csv'

In [ ]:
# 1. Histogramme des scores CVSS
plt.figure()
df['cvss_score'].dropna().hist(bins=20)
plt.title('Distribution des scores CVSS')
plt.xlabel('Score CVSS')
plt.ylabel('Nombre de vulnérabilités')
plt.show()

# 2. Camembert des top 10 CWE
cwe_counts = df['cwe_id'].fillna('Non disponible').value_counts().head(10)
plt.figure()
plt.pie(cwe_counts, labels=cwe_counts.index, autopct='%1.1f%%')
plt.title('Répartition des top 10 CWE')
plt.show()

# 3. Courbe des scores EPSS triés
epss_sorted = df['epss_score'].dropna().sort_values().reset_index(drop=True)
plt.figure()
plt.plot(epss_sorted)
plt.title('Courbe des scores EPSS (triés)')
plt.xlabel('Index des CVE triés')
plt.ylabel('Score EPSS')
plt.show()

# 4. Top 10 des éditeurs les plus touchés
vendor_counts = df['vendor'].value_counts().head(10)
plt.figure()
vendor_counts.plot(kind='bar')
plt.title('Top 10 des éditeurs les plus touchés')
plt.xlabel('Éditeur')
plt.ylabel('Nombre de vulnérabilités')
plt.xticks(rotation=45, ha='right')
plt.show()

# 5. Heatmap de corrélation CVSS vs EPSS
corr = df[['cvss_score','epss_score']].corr()
plt.figure()
plt.imshow(corr, aspect='auto')
plt.colorbar()
plt.xticks([0,1], ['CVSS','EPSS'])
plt.yticks([0,1], ['CVSS','EPSS'])
plt.title('Matrice de corrélation CVSS et EPSS')
plt.show()

# 6. Nuage de points CVSS vs EPSS
plt.figure()
plt.scatter(df['cvss_score'], df['epss_score'])
plt.title('Nuage de points CVSS vs EPSS')
plt.xlabel('Score CVSS')
plt.ylabel('Score EPSS')
plt.show()

# 7. Courbe cumulative des vulnérabilités au fil du temps
df_time = df.dropna(subset=['date_pub']).sort_values('date_pub').copy()
df_time['cum_count'] = range(1, len(df_time) + 1)
plt.figure()
plt.plot(df_time['date_pub'], df_time['cum_count'])
plt.title('Courbe cumulative des vulnérabilités')
plt.xlabel('Date de publication')
plt.ylabel('Nombre cumulatif')
plt.xticks(rotation=45, ha='right')
plt.show()

# 8. Boxplot des scores CVSS par éditeur (Top 10)
top_vendors = vendor_counts.index
df_top = df[df['vendor'].isin(top_vendors)]
plt.figure()
data_to_plot = [df_top[df_top['vendor'] == v]['cvss_score'].dropna() for v in top_vendors]
plt.boxplot(data_to_plot, labels=top_vendors)
plt.title('Dispersion des scores CVSS par top 10 éditeurs')
plt.xlabel('Éditeur')
plt.ylabel('Score CVSS')
plt.xticks(rotation=45, ha='right')
plt.show()

# 9. Top 10 des versions les plus touchées
df_versions = df[['versions']].dropna().copy()
df_versions['versions_list'] = df_versions['versions'].str.split(';')
df_exp = df_versions.explode('versions_list')
df_exp['versions_list'] = df_exp['versions_list'].str.strip()
version_counts = df_exp['versions_list'].value_counts().head(10)
plt.figure()
version_counts.plot(kind='bar')
plt.title('Top 10 des versions les plus touchées')
plt.xlabel('Version')
plt.ylabel('Nombre d'occurrences')
plt.xticks(rotation=45, ha='right')
plt.show()

# 10. Vulnérabilités par éditeur et type de bulletin (avis vs alertes)
df_pivot = df.groupby(['vendor','type_bulletin']).size().unstack(fill_value=0)
df_pivot_top = df_pivot.loc[top_vendors]
plt.figure()
df_pivot_top.plot(kind='bar')
plt.title('Vulnérabilités par éditeur et type de bulletin')
plt.xlabel('Éditeur')
plt.ylabel('Nombre de vulnérabilités')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Type Bulletin')
plt.show()